In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from collections import deque
import ta
import random
import matplotlib.pyplot as plt
import time

In [2]:
Sequential=keras.models.Sequential
Dense,Dropout,LSTM,BatchNormalization=keras.layers.Dense,keras.layers.Dropout,keras.layers.LSTM,keras.layers.BatchNormalization

In [3]:
df=pd.read_csv('GBPUSD_hourly.csv',names=['date','time','open','high','low','close','volume'])

In [4]:
index=df.date+'.'+df.time
df=df.set_index(index)
df=df.drop(['date','time'],1)

In [5]:
df['log_ret']=np.log(df.close/df.close.shift(1))
df['Volatility'] = df['log_ret'].rolling(window=20).std() * np.sqrt(10)
df=df.drop('log_ret',1)

In [6]:

#                  TREND INDICATORS

#                moving average indicator
mov_avr20=ta.trend.EMAIndicator(df.close,n=20)
mov_avr8=ta.trend.EMAIndicator(df.close,n=8)

df['mov_avr20']=mov_avr20.ema_indicator()
df['mov_avr8']=mov_avr8.ema_indicator()

df['bb_mavg']=ta.volatility.bollinger_mavg(df.close,n=20)
df['bb_hband']=ta.volatility.bollinger_hband(df.close,n=20)
df['bb_lband']=ta.volatility.bollinger_lband(df.close,n=20)
#                MACD
macd=ta.trend.macd(df.close)
df['macd']=macd

#               MACD_SIGNAL
macd_signal=ta.trend.macd_signal(df.close)
df['macd_sign']=macd_signal



#                    DAILY RETURN
d_return=ta.others.daily_return(df.close)
df['daily_return']=d_return

#                    DAILY LOG
daily_log=ta.others.daily_log_return(df.close)
df['daily_log']=daily_log

#                    COMMULATIV REUTURN
com_return=ta.others.cumulative_return(df.close)
df['com_return']=com_return


#                   VOLATILITY INDICATORS

#make atr indicator
get=ta.volatility.AverageTrueRange(df.high,df.low,df.close)
df['ATR']=get.average_true_range()

#create keltner band width
kelta=ta.volatility.keltner_channel_wband(df.high,df.low,df.close)
df['kelt_width']=kelta




#                  MOMENTUM INDICATORS

#create williams
will=ta.momentum.wr(df.high,df.low,df.close,lbp=20)
df['will_r']=will

#create awesom oscillator
ao=ta.momentum.ao(df.high,df.low)
df['awesom_osc']=ao

#create roc 
roc=ta.momentum.roc(df.close)
df['roc']=roc

#make rsi indicator
get=ta.momentum.RSIIndicator(df.close,n=20)
df['rsi']=get.rsi()


#                      VOLUME INDICATORS
#create cc_dist
cc_dist=ta.volume.acc_dist_index(df.high,df.low,df.close,df.volume)
df['cc_index']=cc_dist

#chaiking money flow
chaikin_flow=ta.volume.chaikin_money_flow(df.high,df.low,df.close,df.volume)
df['chaiking_flow']=chaikin_flow


#volume price movement
vol_pric=ta.volume.volume_price_trend(df.close,df.volume)
df['vol_price']=vol_pric
#smart money
sm=ta.volume.negative_volume_index(df.close,df.volume)
df['neg_vol']=sm

df=df.dropna()

In [7]:
seq_len=60
future_to_predict=10
Epochs=10
Batch_size=32

In [8]:
df['future']=df['close'].shift(-10)
df['target']=np.where(df['future']>df['close'],1,0)
df=df.drop('future',1)

In [9]:
times=sorted(df.index.values)
last_5pcnt=times[-int(0.1*len(times))]
val_df=df[df.index>=last_5pcnt]
train_df=df[df.index<last_5pcnt]

In [10]:
#Preprocess image to feed to the lstm with shape (batch_len,seq_len,n_feat)

def prepreprocess(df):

    X=scale(df.values[:,:-1])
    Y=df.values[:,-1]
    
        
    df=np.concatenate((X,Y.reshape(-1,1)),1)
    
    
    sequential_data=[]
    prev_days=deque(maxlen=seq_len)
    
    for i in df:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days)==seq_len:
            sequential_data.append([np.array(prev_days),i[-1]])
   
    
    x=[]
    y=[]
    for seq,target in sequential_data:
        x.append(seq)
        y.append(target)
        
    
    return np.array(x),np.array(y)

In [11]:
train_x,train_y=preprocess(train_df)
val_x,val_y=preprocess(val_df)

In [12]:
#lstm implementation

model=Sequential()
model.add(LSTM(units=50,input_shape=(train_x.shape[1],train_x.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=50))
model.add(BatchNormalization())

model.add(Dense(10,activation='linear'))

model.add(Dense(10,activation='tanh'))
model.add(Dense(units=1,activation='sigmoid'))


In [13]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.fit(train_x,train_y,epochs=15,batch_size=Batch_size,validation_data=(val_x,val_y))

In [127]:
predictions=model.predict(val_x)

In [143]:
model.evaluate(val_x,val_y)

334/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.8435097515226124, 0.6317365]